## Download Compustat data from WRDS

-------------------------

### Import libraries

In [1]:
import pandas as pd
import wrds

### Connect WRDS

In [2]:
conn=wrds.Connection()

Enter your WRDS username [yjaey]: yjaeyoon
Enter your password: ···············


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
Loading library list...
Done


### Download funda from Compustat

In [4]:
df= conn.raw_sql(f"""select conm, gvkey, cik, tic, datadate, au, fyear, at, sale,  ni, ib, oancf, sich, prcc_f * csho as mv
                    from comp.funda a
                    
                    where a.fyear >= 1990 and 
                        a.fyear <= 2020 and
                        a.indfmt= 'INDL' and 
                        a.datafmt = 'STD' and 
                        a.popsrc= 'D' and 
                        a.consol = 'C' and
                        (a.sich < 6000 or a.sich > 6999)
                    """)
df= df.sort_values(['gvkey','datadate'])
df['fyear']= df['fyear'].astype(int)
df= df.reset_index(drop=True)
print(df.shape)
df.head()

(211966, 14)


,conm,gvkey,cik,tic,datadate,au,fyear,at,sale,ni,ib,oancf,sich,mv
0,AAR CORP,001004,0000001750,AIR,1991-05-31,6,1990,379.958,466.542,14.801,14.801,36.891,5080.0,224.460360
1,AAR CORP,001004,0000001750,AIR,1992-05-31,6,1991,395.351,422.657,10.020,10.020,8.814,5080.0,204.699595
2,AAR CORP,001004,0000001750,AIR,1993-05-31,6,1992,365.151,382.780,0.283,0.283,16.806,5080.0,214.663477
3,AAR CORP,001004,0000001750,AIR,1994-05-31,6,1993,417.626,407.754,9.494,9.484,6.697,5080.0,228.648720
4,AAR CORP,001004,0000001750,AIR,1995-05-31,6,1994,425.814,451.395,10.463,10.463,15.255,5080.0,243.405235


### Save to .csv

In [5]:
df.to_csv("compustat_from_1990.csv", index= None)

### Close connection

In [6]:
conn.close()